In [0]:
import psycopg2
import numpy as np
import json

In [0]:
#USING PSYCOPG2 JDBC CONNECTOR FOR POSTGRES 
#CONFIGURING GCP POSTGRESQL SERVER TO CONNECT WITH THE SPECIFIC DATABASE

cnx = psycopg2.connect(database="haresh",
                        host="34.30.53.0",
                        user="haresh",
                        password="haresh",
                        port='5432'
                        )
cursor = cnx.cursor()

In [0]:
#DROP ALREADY EXISTING TABLE TO PREVENT OVERWRITING
drop ="DROP TABLE IF EXISTS sample_analytics"
cursor.execute(drop)
cnx.commit()

In [0]:
#CREATE TABLE FOR FINAL TIMESERIES DATA
create_table = """
    CREATE TABLE sample_analytics (
    timestamp_daily TIMESTAMP,
    granularity VARCHAR(20),
    AggregatedName VARCHAR(20),
    Metric FLOAT,
    Dimensions TEXT[]
    )
"""
cursor.execute(create_table)
cnx.commit()

In [0]:
#GET THE TEMPORARY HIVE STORED TABLE 
delta_path = "dbfs:/user/hive/warehouse/timeseries_load"
df = spark.read.format("delta").load(delta_path)
pandas_df = df.toPandas()

In [0]:
#CONVERTING DATAFRAME TO PANDAS
pandas_df = df.toPandas()

In [0]:
#INSERTING THE PANDAS DF TO THE CREATED TABLE
pandas_df['timestamp_daily'] = pandas_df['timestamp_daily'].astype(str)
pandas_df['Granularity'] = pandas_df['Granularity'].astype(str)
pandas_df['AggregatedName'] = pandas_df['AggregatedName'].astype(str)
pandas_df['Metric'] = pandas_df['Metric'].astype(float)
pandas_df['Dimensions'] = pandas_df['Dimensions'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else [])
cols = ",".join(pandas_df.columns.tolist())
for _, row in pandas_df.iterrows():
    dimensions_str = ",".join(row['Dimensions'])
    sql = "INSERT INTO sample_analytics (" + cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row.values))
    cnx.commit()

In [0]:
#SAMPLE QUERY UNCOMMENT TO CHECK 
sql = "SELECT * FROM sample_analytics"
cursor.execute(sql)
result = cursor.fetchall()
for i in result:
    print(i)

(datetime.datetime(2016, 8, 3, 0, 0), 'daily', 'Total visits', 2.0, ['country=country=Russia', 'browser=browser=MRCHROME'])
(datetime.datetime(2016, 8, 2, 0, 0), 'daily', 'Total visits', 878.0, ['country=country=United States', 'browser=browser=Chrome'])
(datetime.datetime(2016, 8, 1, 0, 0), 'daily', 'Total visits', 2.0, ['country=country=Mongolia', 'browser=browser=Safari'])
(datetime.datetime(2016, 8, 1, 0, 0), 'daily', 'Total visits', 3.0, ['country=country=United States', 'browser=browser=Safari (in-app)'])
(datetime.datetime(2016, 8, 3, 0, 0), 'daily', 'Total visits', 1.0, ['country=country=Georgia', 'browser=browser=Safari'])
(datetime.datetime(2016, 8, 3, 0, 0), 'daily', 'Total visits', 1.0, ['country=country=Malta', 'browser=browser=Safari'])
(datetime.datetime(2016, 8, 2, 0, 0), 'daily', 'Total visits', 5.0, ['country=country=Venezuela', 'browser=browser=Chrome'])
(datetime.datetime(2016, 8, 3, 0, 0), 'daily', 'Total visits', 10.0, ['country=country=United States', 'browser=br